<a href="https://colab.research.google.com/github/hschilling/google_colab/blob/master/pandas_in_openmdao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openmdao

     |████████████████████████████████| 4.4MB 1.4MB/s 
  Created wheel for openmdao: filename=openmdao-3.0.0-cp36-none-any.whl size=4626730 sha256=672505259ef46f62257ee6b6af4134c475814f4cf3c1cf162528ec7368fdbe6c
  Stored in directory: /root/.cache/pip/wheels/ad/8f/77/c58ef6cab51fe796172ce215f1c66319164def6edf11c43b88
  Created wheel for pyDOE2: filename=pyDOE2-1.3.0-cp36-none-any.whl size=25519 sha256=2b851d02c31e743f3c478379856f58a0d864ecd66abcf3638385079e15b97962
  Stored in directory: /root/.cache/pip/wheels/60/88/1d/b334ee00dd83e82d111ec56c3be91573d335c93870698037f1
Successfully built openmdao pyDOE2


In [2]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

driver.add_recorder(om.SqliteRecorder("cases.sql"))

driver.recording_options['includes'] = []
driver.recording_options['record_objectives'] = True
driver.recording_options['record_constraints'] = True
driver.recording_options['record_desvars'] = True

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")


Unable to import mpi4py. Parallel processing unavailable.


In [3]:
from collections import defaultdict
case = cr.get_case(0)
outputs = case.list_outputs(out_stream=None)
print(sorted([out[0] for out in outputs]))
varnames = [out[0] for out in outputs]
data = defaultdict(list)
for i in range(len(cr.list_cases())):
  case = cr.get_case(i)
  for varname in varnames:
    data[varname].append(case[varname])

 

['con_cmp1.con1', 'con_cmp2.con2', 'obj_cmp.obj', 'px.x', 'pz.z']


In [0]:
case = cr.get_case(0)
outputs = case.list_outputs(out_stream=None)
varnames = [out[0] for out in outputs]
data = {}
for i in range(len(cr.list_cases())):
  case = cr.get_case(i)
  for varname in varnames:
    if varname not in data:
        data[varname] = []
    data[varname].append(case[varname][0])



In [0]:
def case_recorder_to_data_frame(cr):
  case = cr.get_case(0)
  outputs = case.list_outputs(out_stream=None)
  varnames = [out[0] for out in outputs]
  data = {}
  for i in range(len(cr.list_cases())):
    case = cr.get_case(i)
    for varname in varnames:
      if case[varname].shape == (1,):
        val = case[varname][0]
      else:
        val = np.linalg.norm(case[varname][0])
      if varname not in data:
        data[varname] = []
      data[varname].append(val)
  df = pd.DataFrame(data)
  return df

In [5]:
data

{'con_cmp1.con1': [-22.42830236987513,
  -22.428302369877738,
  -4.574337391558954,
  -0.5507241567968015,
  -0.020433821267987984,
  -3.282107686741753e-05,
  -8.380540705843487e-11],
 'con_cmp2.con2': [-11.94151184938868,
  -11.941511849388423,
  -17.52922772624373,
  -19.947351998016885,
  -20.233245833384178,
  -20.244703769823385,
  -20.2447222330272],
 'obj_cmp.obj': [28.588308165031194,
  28.5883081650338,
  8.56152842495788,
  3.728100457981922,
  3.2035608292750446,
  3.1834263407931056,
  3.183393951723336],
 'px.x': [1.0,
  1.0,
  1.2878587085651816e-14,
  0.0,
  0.0,
  1.1604064831183036e-15,
  0.0],
 'pz.z': [5.0,
  5.0,
  2.864061598967984,
  2.1263240009913784,
  1.9833770833078666,
  1.977648115088326,
  1.9776388834861036]}

In [0]:
import pandas as pd

In [7]:
df = pd.DataFrame(data)
df.drop(['pz.z'], axis=1)

,px.x,obj_cmp.obj,con_cmp1.con1,con_cmp2.con2
0,1.000000e+00,28.588308,-2.242830e+01,-11.941512
1,1.000000e+00,28.588308,-2.242830e+01,-11.941512
2,1.287859e-14,8.561528,-4.574337e+00,-17.529228
3,0.000000e+00,3.728100,-5.507242e-01,-19.947352
4,0.000000e+00,3.203561,-2.043382e-02,-20.233246
5,1.160406e-15,3.183426,-3.282108e-05,-20.244704
6,0.000000e+00,3.183394,-8.380541e-11,-20.244722


In [8]:
df[['con_cmp1.con1']]

,con_cmp1.con1
0,-2.242830e+01
1,-2.242830e+01
2,-4.574337e+00
3,-5.507242e-01
4,-2.043382e-02
5,-3.282108e-05
6,-8.380541e-11


In [11]:
import cufflinks as cf
fig = df[['con_cmp1.con1','px.x']].iplot(asFigure=True)
fig.show()


In [13]:
len(cr.list_cases())

7

In [14]:
df.describe()

,pz.z,px.x,obj_cmp.obj,con_cmp1.con1,con_cmp2.con2
count,7.000000,7.000000e+00,7.000000,7.000000e+00,7.000000
mean,2.989864,2.857143e-01,11.290947,-7.143162e+00,-17.440325
std,1.408606,4.879500e-01,11.971591,1.056813e+01,3.879359
min,1.977639,0.000000e+00,3.183394,-2.242830e+01,-20.244722
25%,1.980513,0.000000e+00,3.193494,-1.350132e+01,-20.238975
50%,2.126324,1.160406e-15,3.728100,-5.507242e-01,-19.947352
75%,3.932031,5.000000e-01,18.574918,-1.023332e-02,-14.735370
max,5.000000,1.000000e+00,28.588308,-8.380541e-11,-11.941512


In [15]:
df.head()

,pz.z,px.x,obj_cmp.obj,con_cmp1.con1,con_cmp2.con2
0,5.000000,1.000000e+00,28.588308,-22.428302,-11.941512
1,5.000000,1.000000e+00,28.588308,-22.428302,-11.941512
2,2.864062,1.287859e-14,8.561528,-4.574337,-17.529228
3,2.126324,0.000000e+00,3.728100,-0.550724,-19.947352
4,1.983377,0.000000e+00,3.203561,-0.020434,-20.233246


In [35]:
df_from_cr = case_recorder_to_data_frame(cr)

(2,)
(1,)
(1,)
(1,)
(1,)
(2,)
(1,)
(1,)
(1,)
(1,)
(2,)
(1,)
(1,)
(1,)
(1,)
(2,)
(1,)
(1,)
(1,)
(1,)
(2,)
(1,)
(1,)
(1,)
(1,)
(2,)
(1,)
(1,)
(1,)
(1,)
(2,)
(1,)
(1,)
(1,)
(1,)


In [36]:
df_from_cr.columns

Index(['pz.z', 'px.x', 'obj_cmp.obj', 'con_cmp1.con1', 'con_cmp2.con2'], dtype='object')

In [37]:
fig = df_from_cr[['con_cmp1.con1','px.x']].iplot(asFigure=True)
fig.show()


In [38]:
fig = df_from_cr.iplot(asFigure=True)
fig.show()


In [0]:
a = np.arange(9) - 4

In [0]:
b = a.reshape((3, 3))

In [0]:
from numpy import linalg as LA

In [25]:
LA.norm(a)

7.745966692414834

In [26]:
c = np.arange(3)
LA.norm(c)

2.23606797749979

In [0]:
d = [1,1,1,1]

In [32]:
LA.norm(d)

2.0